## Film ID for calculations 302 (Вариант 37)

In [11]:
! pwd

/data/home/yuriy.perevezentsev


In [12]:
! chmod 0600 newprolab.pem

In [9]:
! ssh -i newprolab.pem yuriy.perevezentsev@spark-master-5.newprolab.com

The authenticity of host 'spark-master-5.newprolab.com (10.0.0.5)' can't be established.
ECDSA key fingerprint is SHA256:AMs7xhjQ6SOj3WQYjee8DRnIt2R6iigasdxzV4o9f64.
Are you sure you want to continue connecting (yes/no)? ^C


In [13]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python' 
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'  

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip')) 
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [14]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

# В драйвере создается SparkContext/SparkSession

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [15]:
spark

In [45]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [105]:
# !hdfs dfs -cat /labs/laba01/ml-100k/u.data

In [62]:
# Selecting necessary columns
rdd_all_films = sc.textFile("/labs/laba01/ml-100k/u.data").map(lambda x: (x.split("\t")[2],1))
rdd_all_films.take(10)

[('3', 1),
 ('3', 1),
 ('1', 1),
 ('2', 1),
 ('1', 1),
 ('4', 1),
 ('2', 1),
 ('5', 1),
 ('3', 1),
 ('3', 1)]

In [79]:
# Counting ratings for all films
answer_all = rdd_all_films.groupByKey().map(lambda x: (x[0], len(x[1]))).collect()
answer_all.sort(key=lambda i:i[0],reverse = False)
print(answer_all)

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]


In [89]:
# Counting ratings for film 302
rdd_film302 = (
    sc.textFile("/labs/laba01/ml-100k/u.data")
               .map(lambda x: (x.split("\t")))
               .filter(lambda x: x[1] == "302")
               .map(lambda x: (x[2], 1))
              )
rdd_film302.take(10)

[('3', 1),
 ('4', 1),
 ('4', 1),
 ('4', 1),
 ('3', 1),
 ('5', 1),
 ('3', 1),
 ('4', 1),
 ('5', 1),
 ('4', 1)]

In [90]:
answer_302 = rdd_film302.groupByKey().map(lambda x: (x[0], len(x[1]))).collect()
answer_302.sort(key=lambda i:i[0],reverse = False)
print(answer_302)

[('1', 2), ('2', 10), ('3', 46), ('4', 119), ('5', 120)]


In [98]:
# Combining
hist_all = [i[1] for i in answer_all]
print(hist_all)

[6110, 11370, 27145, 34174, 21201]


In [101]:
hist_film = [i[1] for i in answer_302]
print(hist_film)

[2, 10, 46, 119, 120]


In [103]:
final_answer: dct = {
    'hist_film' : hist_film, 
    'hist_all' : hist_all
}
with open("lab01.json", "w") as outfile:
    json.dump(final_answer, outfile)

In [104]:
sc.stop()